In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluatePortfolioPerformance import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.generatePortfolioAction import *
from AI_Trading.src.train import *
from AI_Trading.src.customizedEnv import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import pandas as pd
from tqdm import tqdm
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
#evaluate setting
exp = 'test'
#調整對應檔案路徑
acitonFile_path = f'/Users/shrshiunhuang/Documents/Project/FinRL/AI_Trading/results/{exp}/'
model_name = 'PPO'
save_path = os.path.join(config.EVALUATE_RESULT_PATH, exp)
if not os.path.isdir(save_path):
    try:
        os.mkdir(save_path)
    except Exception:
        print(f'no folder {save_path}')
        pass

In [ ]:
dir_list = [config.RESULTS_DIR]
for path in dir_list:
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except Exception:
            print(f'no folder {path}')
            pass

In [ ]:
# (with total revenu)
df_annualReturn_allYear = pd.DataFrame()
df_mdd_allYear = pd.DataFrame()
df_sharpe_allYear = pd.DataFrame()
df_sortino_allYear = pd.DataFrame()
df_calmar_allYear = pd.DataFrame()

df_annualReturn_rank_allYear = pd.DataFrame()
df_mdd_rank_allYear = pd.DataFrame()
df_sharpe_rank_allYear = pd.DataFrame()
df_sortino_rank_allYear = pd.DataFrame()
df_calmar_rank_allYear = pd.DataFrame()

df_action_all = pd.DataFrame()
df_trade_all = pd.DataFrame()

df_action_all_eq = pd.DataFrame()
df_trade_all_eq = pd.DataFrame()

df_action_all_82 = pd.DataFrame()
df_trade_all_82 = pd.DataFrame()

df_action_all_mv = pd.DataFrame()
df_trade_all_mv = pd.DataFrame()

df_action_all_stock = pd.DataFrame()
df_action_all_debt = pd.DataFrame()
df_action_all_reit = pd.DataFrame()

change_threshold_list = [0.3, 0.9, 0.6, 0.7, 0.8, 0.4, 0.8, 0.8, 0.8, 0.8, 0.6, 0.8, 0.9, 0.7, 0.9]
# evaluate
for i in tqdm(range(len(config.TEST_START_DATE))):
        train,trade = preprocess(config.TRAIN_START_DATE[i], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i],cov=True, adjClose=True)

        # load actions file
        df_actions = pd.read_csv(f'{acitonFile_path}df_action_{model_name}_{i}.csv')
        if((df_actions.columns == config.DF_ACTION_ORDER).any()):
            # weight 變動總和超過 change_threshold 才做變動(default 0)
            df_return, df_portfolio_value = computeReturns(df_actions,trade, transCostRate=0.001)
            returns, stats = getStats(df_return)

            # weight 變動總和超過 change_threshold 才做變動(依 val 決定)
            df_return2, df_portfolio_value2 = computeReturns(df_actions,trade, change_threshold=change_threshold_list[i], transCostRate=0.001)
            returns2, stats2 = getStats(df_return2)

            # weight 變動總和超過 change_threshold 才做變動(60%)
            df_return3, df_portfolio_value3 = computeReturns(df_actions,trade, change_threshold=0.55, transCostRate=0.001)
            returns3, stats3 = getStats(df_return3)

            # Equal weight
            equalWeight_actions = getScaleWeightActions(trade, stock=3.3333334, debt=3.3333333, reit=3.3333333)
            df_equalWeight_return, df_equalWeight_portfolio_value = computeReturns(equalWeight_actions,trade, transCostRate=0.001, rebalance=False)
            equalWeight_returns, equalWeight_stats = getStats(df_equalWeight_return)

            #stock(8) debt(2)
            scaleWeight82_actions = getScaleWeightActions(trade, stock=8, debt=2)
            df_scaleWeight82_return, df_scaleWeight82_portfolio_value = computeReturns(scaleWeight82_actions,trade, transCostRate=0.001, change_threshold=0, rebalance=False)
            scaleWeight82_returns, scaleWeight82_stats = getStats(df_scaleWeight82_return)

            # # stock(7) debt(3)
            # scaleWeight73_actions = getScaleWeightActions(trade, stock=7, debt=3)
            # df_scaleWeight73_return, df_scaleWeight73_portfolio_value = computeReturns(scaleWeight73_actions,trade, transCostRate=0.001)
            # scaleWeight73_returns, scaleWeight73_stats = getStats(df_scaleWeight73_return)

            # # stock(6) debt(4)
            # scaleWeight64_actions = getScaleWeightActions(trade, stock=6, debt=4)
            # df_scaleWeight64_return, df_scaleWeight64_portfolio_value = computeReturns(scaleWeight64_actions,trade, transCostRate=0.001)
            # scaleWeight64_returns, scaleWeight64_stats = getStats(df_scaleWeight64_return)

            # Min-Variance
            minVariance_actions = getMinVarianceActions(trade)
            # minVariance_actions.to_csv(f'{acitonFile_path}/minVariance/df_action_DDPG_{i}.csv')
            df_minVariance_return, df_minVariance_portfolio_value = computeReturns(minVariance_actions,trade, transCostRate=0.001)
            minVariance_returns, minVariance_stats = getStats(df_minVariance_return)

            # # MaxSharpe
            # maxSharpe_actions = getMaxSharpeActions(trade)
            # # maxSharpe_actions.to_csv(f'{acitonFile_path}/maxSharpe/df_action_DDPG_{i}.csv')
            # df_maxSharpe_return, df_maxSharpe_portfolio_value = computeReturns(maxSharpe_actions,trade, transCostRate=0.001)
            # maxSharpe_returns, maxSharpe_stats = getStats(df_maxSharpe_return)

            # All In Stock
            all_stock_actions= getScaleWeightActions(trade, stock=1)
            df_stock_return, df_stock_value = computeReturns(all_stock_actions,trade, transCostRate=0.001)
            stock_returns, stock_stats = getStats(df_stock_return)

            # All In Debt
            all_debt_actions= getScaleWeightActions(trade, debt=1)
            df_debt_return, df_debt_value = computeReturns(all_debt_actions,trade, transCostRate=0.001)
            debt_returns, debt_stats = getStats(df_debt_return)

            # All In Reit
            all_reit_actions= getScaleWeightActions(trade, reit=1)
            df_reit_return, df_reit_value = computeReturns(all_reit_actions,trade, transCostRate=0.001)
            reit_returns, reit_stats = getStats(df_reit_return)
            
            df_stats = pd.concat({
                                'Ours': stats,
                                'threshold': stats2,
                                '60%': stats3,
                                'equalWeight': equalWeight_stats,
                                'stock(8)debt(2)': scaleWeight82_stats,
                                # 'stock(7)debt(3)': scaleWeight73_stats,
                                # 'stock(6)debt(4)': scaleWeight64_stats,
                                'minVariance':minVariance_stats,
                                # 'maxSharpe':maxSharpe_stats,
                                'stock': stock_stats,
                                'debt': debt_stats,
                                'reit': reit_stats
                                },axis=1)

            # get each year stats
            df_annualReturn_allYear = stats_allYear(i, 'Annual return', df_stats, df_annualReturn_allYear)
            df_mdd_allYear = stats_allYear(i, 'Max drawdown', df_stats, df_mdd_allYear)
            df_sharpe_allYear = stats_allYear(i, 'Sharpe ratio', df_stats, df_sharpe_allYear)
            df_sortino_allYear = stats_allYear(i, 'Sortino ratio', df_stats, df_sortino_allYear)           
            df_calmar_allYear = stats_allYear(i, 'Calmar ratio', df_stats, df_calmar_allYear)

            # get each rank
            # df_annualReturn_rank_allYear = rankCaculate(i, 'Annual return', df_stats, df_annualReturn_rank_allYear)
            # df_mdd_rank_allYear = rankCaculate(i, 'Max drawdown', df_stats, df_mdd_rank_allYear)
            # df_sharpe_rank_allYear = rankCaculate(i, 'Sharpe ratio', df_stats, df_sharpe_rank_allYear)
            # df_sortino_rank_allYear = rankCaculate(i, 'Sortino ratio', df_stats, df_sortino_rank_allYear)
            # df_calmar_rank_allYear = rankCaculate(i, 'Calmar ratio', df_stats, df_calmar_rank_allYear)

            action = df_actions
            action_eq = equalWeight_actions
            action_82 = scaleWeight82_actions
            action_mv =  minVariance_actions
            action_stock =  all_stock_actions
            action_debt =  all_debt_actions
            action_reit =  all_reit_actions

            # padding
            if i > 0:
                action.TLT.iloc[0] = lastWeight_TLT
                action.VNQ.iloc[0] = lastWeight_VNQ
                action.VTI.iloc[0] = lastWeight_VTI
                
                action_eq.TLT.iloc[0] = lastWeight_TLT_eq
                action_eq.VNQ.iloc[0] = lastWeight_VNQ_eq
                action_eq.VTI.iloc[0] = lastWeight_VTI_eq

                action_82.TLT.iloc[0] = lastWeight_TLT_82
                action_82.VNQ.iloc[0] = lastWeight_VNQ_82
                action_82.VTI.iloc[0] = lastWeight_VTI_82

                action_mv.TLT.iloc[0] = lastWeight_TLT_mv
                action_mv.VNQ.iloc[0] = lastWeight_VNQ_mv
                action_mv.VTI.iloc[0] = lastWeight_VTI_mv

                action_debt.TLT.iloc[0] = lastWeight_TLT_debt
                action_reit.VNQ.iloc[0] = lastWeight_VNQ_reit
                action_stock.VTI.iloc[0] = lastWeight_VTI_stock

            lastWeight_TLT = action.TLT.iloc[-1]
            lastWeight_VNQ = action.VNQ.iloc[-1]
            lastWeight_VTI = action.VTI.iloc[-1]

            lastWeight_TLT_eq = action_eq.TLT.iloc[-1]
            lastWeight_VNQ_eq = action_eq.VNQ.iloc[-1]
            lastWeight_VTI_eq = action_eq.VTI.iloc[-1]

            lastWeight_TLT_82 = action_82.TLT.iloc[-1]
            lastWeight_VNQ_82 = action_82.VNQ.iloc[-1]
            lastWeight_VTI_82 = action_82.VTI.iloc[-1]

            lastWeight_TLT_mv = action_mv.TLT.iloc[-1]
            lastWeight_VNQ_mv = action_mv.VNQ.iloc[-1]
            lastWeight_VTI_mv = action_mv.VTI.iloc[-1]

            lastWeight_TLT_debt = action_debt.TLT.iloc[-1]
            lastWeight_VNQ_reit = action_reit.VNQ.iloc[-1]
            lastWeight_VTI_stock = action_stock.VTI.iloc[-1]
            
            df_action_all = pd.concat([df_action_all,action],axis=0,ignore_index=True)
            df_trade_all = pd.concat([df_trade_all,trade],axis=0,ignore_index=True)

            df_action_all_eq = pd.concat([df_action_all_eq,action_eq],axis=0,ignore_index=True)
            df_trade_all_eq = pd.concat([df_trade_all_eq,trade],axis=0,ignore_index=True)

            df_action_all_82 = pd.concat([df_action_all_82,action_82],axis=0,ignore_index=True)
            df_trade_all_82 = pd.concat([df_trade_all_82,trade],axis=0,ignore_index=True)

            df_action_all_mv = pd.concat([df_action_all_mv,action_mv],axis=0,ignore_index=True)
            df_trade_all_mv = pd.concat([df_trade_all_mv,trade],axis=0,ignore_index=True)

            df_action_all_stock = pd.concat([df_action_all_stock,action_stock],axis=0,ignore_index=True)
            df_action_all_debt = pd.concat([df_action_all_debt,action_debt],axis=0,ignore_index=True)
            df_action_all_reit = pd.concat([df_action_all_reit,action_reit],axis=0,ignore_index=True)
        else:
            print('order of title is wrong!')

In [ ]:
df_return_all, df_portfolio_value_all = computeReturns(df_action_all,df_trade_all, transCostRate=0.001, change_threshold=0, rebalance=False)
returns_all, stats_all = getStats(df_return_all)
print(df_portfolio_value_all[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all, df_portfolio_value_all = computeReturns(df_action_all,df_trade_all, transCostRate=0, change_threshold=0, rebalance=False)
returns_all, stats_all = getStats(df_return_all)
print('threshold:0 ; transCost=0')
print(df_portfolio_value_all[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_ct, df_portfolio_value_all = computeReturns(df_action_all,df_trade_all, transCostRate=0.001, change_threshold=0.55)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_portfolio_value_all.to_csv(f'{acitonFile_path}/df_portfolio_value_DDPG_all_from2010.csv')
returns_all_ct, stats_all = getStats(df_return_all_ct)
print(df_portfolio_value_all[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_ct, df_portfolio_value_all = computeReturns(df_action_all,df_trade_all, transCostRate=0, change_threshold=0.6)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_portfolio_value_all.to_csv(f'{acitonFile_path}/df_portfolio_value_DDPG_all_from2010.csv')
returns_all_ct, stats_all = getStats(df_return_all_ct)
print('threshold:0.6 ; transCost=0')
print(df_portfolio_value_all[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_eq, df_equalWeight_portfolio_value = computeReturns(df_action_all_eq,df_trade_all_eq, transCostRate=0, rebalance=True)
df_action_all_eq.to_csv(f'{acitonFile_path}/df_action_DDPG_eq.csv')
df_equalWeight_portfolio_value.to_csv(f'{acitonFile_path}/df_portfolio_value_DDPG_all_eq.csv')
returns_all_eq, stats_all = getStats(df_return_all_eq)
print(df_equalWeight_portfolio_value[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_mv, df_minVariance_portfolio_value = computeReturns(df_action_all_mv,df_trade_all, transCostRate=0)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_minVariance_portfolio_value.to_csv(f'{acitonFile_path}/df_portfolio_value_scaleWeight82_all_from2010.csv')
returns_all_mv, stats_all = getStats(df_return_all_mv)
print(df_minVariance_portfolio_value[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_82, df_scaleWeight82_portfolio_value = computeReturns(df_action_all_82,df_trade_all_82, transCostRate=0, rebalance=True)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_scaleWeight82_portfolio_value.to_csv(f'{acitonFile_path}/df_portfolio_value_scaleWeight82_all_from2010.csv')
returns_all_82, stats_all = getStats(df_return_all_82)
print(df_scaleWeight82_portfolio_value[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_stock, df_scaleWeight_stock_portfolio_value = computeReturns(df_action_all_stock,df_trade_all, transCostRate=0.01, rebalance=False)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_scaleWeight82_portfolio_value.to_csv(f'{acitonFile_path}/df_portfolio_value_scaleWeight82_all_from2010.csv')
returns_all_stock, stats_all = getStats(df_return_all_stock)
print(df_scaleWeight_stock_portfolio_value[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_debt, df_scaleWeight_debt_portfolio_value = computeReturns(df_action_all_debt,df_trade_all, transCostRate=0, rebalance=False)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_scaleWeight82_portfolio_value.to_csv(f'{acitonFile_path}/df_portfolio_value_scaleWeight82_all_from2010.csv')
returns_all_debt, stats_all = getStats(df_return_all_debt)
print(df_scaleWeight_debt_portfolio_value[-1:])
pd.DataFrame(stats_all)

In [ ]:
df_return_all_reit, df_scaleWeight_reit_portfolio_value = computeReturns(df_action_all_reit,df_trade_all, transCostRate=0, rebalance=False)
# df_action_all.to_csv(f'{acitonFile_path}/df_action_DDPG_all.csv')
# df_scaleWeight82_portfolio_value.to_csv(f'{acitonFile_path}/df_portfolio_value_scaleWeight82_all_from2010.csv')
returns_all_reit, stats_all = getStats(df_return_all_reit)
print(df_scaleWeight_reit_portfolio_value[-1:])
df_stats =pd.DataFrame(stats_all)


In [ ]:
df_annualReturn_allYear_avg = average_allYear(df_annualReturn_allYear)
df_annualReturn_allYear_avg.to_csv(f'{save_path}/df_annualReturn.csv')
df_annualReturn_allYear_avg

In [ ]:
df_mdd_allYear_avg = average_allYear(df_mdd_allYear)
df_mdd_allYear_avg.to_csv(f'{save_path}/df_mdd.csv')
df_mdd_allYear_avg

In [ ]:
df_calmar_allYear_avg = average_allYear(df_calmar_allYear)
df_calmar_allYear_avg.to_csv(f'{save_path}/df_calmar.csv')
df_calmar_allYear_avg

In [ ]:
df_sharpe_allYear_avg = average_allYear(df_sharpe_allYear)
df_sharpe_allYear_avg.to_csv(f'{save_path}/df_sharpe.csv')
df_sharpe_allYear_avg

In [ ]:
df_sortino_allYear_avg = average_allYear(df_sortino_allYear)
df_sortino_allYear_avg.to_csv(f'{save_path}/df_sortino.csv')
df_sortino_allYear_avg

In [ ]:
df_annualReturn_rank_allYear_avg = average_allYear(df_annualReturn_rank_allYear)
df_annualReturn_rank_allYear_avg

In [ ]:
df_mdd_rank_allYear_avg  = average_allYear(df_mdd_rank_allYear)
df_mdd_rank_allYear_avg 

In [ ]:
df_sharpe_rank_allYear_avg = average_allYear(df_sharpe_rank_allYear)
df_sharpe_rank_allYear_avg 

In [ ]:
df_sortino_rank_allYear_avg = average_allYear(df_sortino_rank_allYear)
df_sortino_rank_allYear_avg

In [ ]:
df_calmar_rank_allYear_avg = average_allYear(df_calmar_rank_allYear)
df_calmar_rank_allYear_avg

In [ ]:
df_select = pd.DataFrame(columns = ['year', 'DDPG', 'Equal' , 'MinVar', '8:2'])
for year in range(2008, 2022, 1):
    year_begin = str(year)+'-01-01'
    year_end = str(year+1)+'-01-01'
    select_ddpg_pv = df_portfolio_value_all.loc[(df_portfolio_value_all['date']>=year_begin)&(df_portfolio_value_all['date']<year_end), 'portfolio_value']
    select_equal_pv = df_equalWeight_portfolio_value.loc[(df_equalWeight_portfolio_value['date']>=year_begin)&(df_equalWeight_portfolio_value['date']<year_end), 'portfolio_value']
    select_minvar_pv = df_minVariance_portfolio_value.loc[(df_minVariance_portfolio_value['date']>=year_begin)&(df_minVariance_portfolio_value['date']<year_end), 'portfolio_value']
    select_82_pv = df_scaleWeight82_portfolio_value.loc[(df_scaleWeight82_portfolio_value['date']>=year_begin)&(df_scaleWeight82_portfolio_value['date']<year_end), 'portfolio_value']
    
    select_ddpg_return = (select_ddpg_pv.iloc[-1] - select_ddpg_pv.iloc[0])/select_ddpg_pv.iloc[0]
    select_equal_return = (select_equal_pv.iloc[-1] - select_equal_pv.iloc[0])/select_equal_pv.iloc[0]
    select_minvar_return = (select_minvar_pv.iloc[-1] - select_minvar_pv.iloc[0])/select_minvar_pv.iloc[0]
    select_82_return = (select_82_pv.iloc[-1] - select_82_pv.iloc[0])/select_82_pv.iloc[0]
    
    df_select = df_select.append({'year':str(year), 'DDPG': select_ddpg_return, 'Equal': select_equal_return, 'MinVar': select_minvar_return, '8:2':select_82_return}, ignore_index=True)

In [ ]:
VTI = pd.read_csv('./AI_Trading/data/VTI.csv')
TLT = pd.read_csv('./AI_Trading/data/TLT.csv')
VNQ = pd.read_csv('./AI_Trading/data/VNQ.csv')
weight_price_plot(exp, df_action_all, VTI, TLT, VNQ, rolling_window=10, time_period = 180, price_return=False)

In [ ]:
cumulativeReturnPlot(returns_all, returns_all_82, returns_all_eq, returns_all_mv, returns_all_stock, returns_all_debt, returns_all_reit)

In [ ]:
import matplotlib.pyplot as plt
fig,ax0 = plt.subplots()
alpha = ['base', '1', '10', '100', '500', '1000']
avg = [0.898, 1.487, 1.848, 1.587, 2.364, 1.842]
median = [1.505, 0.711, 1.688, 0.789, 0.924, 0.093]
culmulate = [0.282352, 0.062745, 0.142813, 0.091681, 0.131873, 0.208957]
calmar = [0, 34878.94, 24041.08, 61979.18, 31523.75, 87333]
x = range(6)
plt.xticks(ticks=x, labels=alpha)
ax0.bar(x,calmar, color=['lightgray'])
ax0.set_ylabel('sum(calmar)')
ax1 = ax0.twinx()
ax1.plot(x,avg,'r')
ax1.plot(x,median, 'g')
ax1.plot(x,culmulate, 'orange')
ax1.legend(["Avgerage", "Median", "cumulated", "calmar"])
ax1.set_ylabel('calmar')
plt.title('clamar')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig,ax0 = plt.subplots()
alpha = ['base', '0.1', '1', '10', '100', '500', '1000']
avg = [-0.133, -0.130, -0.114, -0.107, -0.103, -0.107, -0.136]
median = [-0.128, -0.100, -0.075, -0.068, -0.079, -0.079, -0.100]
culmulate = [-0.320818, -0.366038, -0.318437, -0.318443, -0.2925, -0.302868, -0.438215]
mdd = [0, -125.37, -103.80, -99.39, -97.31, -99.91, -129.36]
x = range(7)
plt.xticks(ticks=x, labels=alpha)
ax0.bar(x,mdd, color=['lightgray'])
ax0.set_ylabel('sum(mdd)')
ax1 = ax0.twinx()
ax1.plot(x,avg,'r')
ax1.plot(x,median, 'g')
ax1.plot(x,culmulate, 'orange')
ax1.legend(["Avgerage", "Median", "cumulated", "calmar"])
ax1.set_ylabel('mdd')
plt.title('mdd')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig,ax0 = plt.subplots()
alpha = ['base', '-1', '-10', '-100', '-500', '-1000']
avg = [-0.133,  -0.143, -0.111, -0.118, -0.107, -0.113]
median = [-0.128, -0.113, -0.079, -0.852, -0.079, -0.068]
culmulate = [-0.320818, -0.469703, -0.336283, -0.327378, -0.292502, -0.318443]
var = [0, 0.023264, 0.0165686, 0.018773, 0.015510, 0.014889]
x = range(6)
plt.xticks(ticks=x, labels=alpha)
ax0.bar(x, var, color=['lightgray'])
ax0.set_ylabel('mean(var)')
ax1 = ax0.twinx()
ax1.plot(x,avg,'r')
ax1.plot(x,median, 'g')
ax1.plot(x,culmulate, 'orange')
ax1.legend(["Avgerage", "Median", "cumulated", "calmar"])
ax1.set_ylabel('mdd')
plt.title('risk(volaitility)')

plt.show()